In [3]:
import pandas as pd 

df_1line = pd.read_csv('../DATA/국가철도공단_수도권1호선_역위치_20231106.csv',encoding='cp949')
df_1line

,철도운영기관명,선명,역명,경도,위도
0,코레일,1호선,지행,127.055734,37.892361
1,코레일,1호선,도봉,NaN,NaN
2,코레일,1호선,창동,127.047663,37.653181
3,코레일,1호선,월계,127.058835,37.633114
4,코레일,1호선,외대앞,127.063572,37.596157
...,...,...,...,...,...
94,서울교통공사,1호선,동대문,127.007890,37.565145
95,서울교통공사,1호선,신설동,127.024710,37.576117
96,서울교통공사,1호선,제기동,127.034902,37.578116
97,서울교통공사,1호선,청량리(서울시립대입구),127.045063,37.580148


In [5]:
df_filtered = df_1line[pd.isna(df_1line['경도'])]
df_filtered

,철도운영기관명,선명,역명,경도,위도
1,코레일,1호선,도봉,NaN,NaN
16,코레일,1호선,보산,NaN,NaN
52,코레일,1호선,광운대,NaN,NaN


In [6]:
# google map 검색
# url 주소 스플릿 해야 하는데?

# !3d37.679441!4d127.045521!
# !3d37.914277!4d127.057158!
# !3d37.6234616!4d127.0615573!

#coor_list = [37.679441!4d127.045521,37.914277!4d127.057158,37.6234616!4d127.0615573]

lat_list = [37.679441,37.914277,37.6234616]  # 위도 리스트
lng_list = [127.045521,127.057158,127.0615573]  # 경도 리스트

# for coordinate in coor_list:
#     # '!'를 기준으로 문자열을 분리
#     lat, lng = coordinate.split('!4d')
#     # 위도와 경도를 각각의 리스트에 추가
#     lat_list.append(float(lat))
#     lng_list.append(float(lng.split('d')[1]))  # 'd'를 기준으로 나눈 두 번째 요소가 경도
    
# lat

In [8]:
df_filtered['위도'] = lat_list
df_filtered['경도'] = lng_list
df_filtered

/tmp/ipykernel_16992/4168695146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['위도'] = lat_list
/tmp/ipykernel_16992/4168695146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['경도'] = lng_list


,철도운영기관명,선명,역명,경도,위도
1,코레일,1호선,도봉,127.045521,37.679441
16,코레일,1호선,보산,127.057158,37.914277
52,코레일,1호선,광운대,127.061557,37.623462


In [10]:
#df1과 df2가 이름을 공유하는 열에서 덮어쓰기

df_1line.update(df_filtered, overwrite=True)
df_1line.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   철도운영기관명  99 non-null     object 
 1   선명       99 non-null     object 
 2   역명       99 non-null     object 
 3   경도       99 non-null     float64
 4   위도       99 non-null     float64
dtypes: float64(2), object(3)
memory usage: 4.0+ KB


In [12]:

df = pd.DataFrame({'역명': df_1line['역명'],
                   '호선': df_1line['선명'],
                   '위도': df_1line['위도'],
                   '경도': df_1line['경도']})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역명      99 non-null     object 
 1   호선      99 non-null     object 
 2   위도      99 non-null     float64
 3   경도      99 non-null     float64
dtypes: float64(2), object(2)
memory usage: 3.2+ KB


In [14]:
import mysql.connector

remote = mysql.connector.connect(
    host = 'database-1.c3ieyo8m4v9o.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'eda_team',
    database = 'eda_project',
    password = '123456'
)

cur = remote.cursor()
cur.execute("CREATE TABLE seoul_metro_1line_coor(역명 VARCHAR(128), 호선 VARCHAR(128), latitude double, longitude double)")


In [15]:
cursor = remote.cursor(buffered=True)

sql = "INSERT INTO seoul_metro_1line_coor VALUES(%s,%s,%s,%s)"

for i, row in df.iterrows():
    cursor.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit() # 데이터베이스에 적용하기 위한 명령


('지행', '1호선', 37.892361, 127.055734)
('도봉', '1호선', 37.679441, 127.045521)
('창동', '1호선', 37.653181, 127.047663)
('월계', '1호선', 37.633114, 127.058835)
('외대앞', '1호선', 37.596157, 127.063572)
('대방', '1호선', 37.513392, 126.926407)
('신길', '1호선', 37.517018, 126.917743)
('덕정', '1호선', 37.843216, 127.061511)
('구일', '1호선', 37.496273, 126.869599)
('온수', '1호선', 37.492267, 126.823399)
('역곡', '1호선', 37.48514, 126.811622)
('송내', '1호선', 37.487626, 126.753045)
('부평', '1호선', 37.489457, 126.724584)
('성균관대', '1호선', 37.300344, 126.970861)
('소요산', '1호선', 37.5653, 127.0339)
('동두천', '1호선', 37.927569, 127.054825)
('보산', '1호선', 37.914277, 127.057158)
('동두천중앙', '1호선', 37.901818, 127.056528)
('덕계', '1호선', 37.818761, 127.056676)
('녹양', '1호선', 37.759416, 127.042272)
('가능', '1호선', 37.748362, 127.044251)
('의정부', '1호선', 37.73873, 127.045891)
('회룡', '1호선', 37.724037, 127.047319)
('망월사', '1호선', 37.710499, 127.047361)
('도봉산', '1호선', 37.689603, 127.046347)
('방학', '1호선', 37.66796, 127.04456)
('녹천', '1호선', 37.644599, 127.051406